# Modeling: Подготовка датасетов для моделей и создание самих моделей

## 1. Импорт библиотек

Импортируем стандартные библиотеки для анализа данных, а также пользовательские функции для визуализации и первичного осмотра данных.

In [119]:
import pandas as pd

from sklearn.model_selection import train_test_split

try:
    from utils.cleaning import normalize_day_of_week # noqa
    from utils.paths import HOME, pj # noqa
except:
    from scooter_sharing_analysis.utils.paths import HOME, pj
    from scooter_sharing_analysis.utils.cleaning import normalize_day_of_week

## 2. Загрузка очищенных данных

Загружаем файл cleaned_rides.csv нормализованный после очистки.

In [120]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": pj("data", "cleaned_rides.csv"),
    "encoding": "utf-8",
    "parse_dates": ["start_date", "end_date", "day_timestamp", "hour_timestamp"],
}

rides_data = pd.read_csv(**rides_options)

## 3. Создание факторов для моделей: временные, лаговые, погодные

## 3.1. Создание временных и лаговых факторов

Группируем данные по часовому ряду, получая спрос за каждый час (demand), добавляем новые колонки: 'hour_of_day', 'day_of_week', 'day_of_week_type'. Делим данные на тестовые и тренировочные с соотношение 1:5, добавляем лаговые колонки: 'lag_1h', 'lag_24h', 'mean_last_24h', 'mean_last_7d'

In [121]:
# Группировка данных и получение спроса
demand = rides_data.groupby("hour_timestamp").agg({"id": "count"}).rename(columns={"id": "demand"}).reset_index()

# Добавление новых колонок на основе "hour_timestamp"
demand["hour_of_day"] = demand["hour_timestamp"].dt.hour
demand["day_of_week"] = demand["hour_timestamp"].dt.dayofweek
demand["day_of_week_type"] = demand["day_of_week"].apply(normalize_day_of_week)

# Разбиение данных на тестувую и тренировочную часть
X_demand_test, X_demand_train, Y_demand_test, Y_demand_train = train_test_split(demand, demand["demand"], test_size=0.2, shuffle=False)

# Создание лаговых признаков
X_demand_test["lag_1h"] = X_demand_test["demand"].shift(1)
X_demand_test["lag_24h"] = X_demand_test["demand"].shift(24)
X_demand_test["mean_last_24h"] = X_demand_test["lag_1h"].rolling(24).mean()
X_demand_test["mean_last_7d"] = X_demand_test["lag_1h"].rolling(24 * 7).mean()

# Вывод первой 1000 строк
X_demand_test.head(1000)


,hour_timestamp,demand,hour_of_day,day_of_week,day_of_week_type,lag_1h,lag_24h,mean_last_24h,mean_last_7d
0,2023-04-22 09:00:00,19,9,5,суббота,NaN,NaN,NaN,NaN
1,2023-04-22 10:00:00,41,10,5,суббота,19.0,NaN,NaN,NaN
2,2023-04-22 11:00:00,27,11,5,суббота,41.0,NaN,NaN,NaN
3,2023-04-22 12:00:00,42,12,5,суббота,27.0,NaN,NaN,NaN
4,2023-04-22 13:00:00,46,13,5,суббота,42.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,2023-06-02 22:00:00,74,22,4,пятница,89.0,32.0,38.916667,39.648810
996,2023-06-02 23:00:00,29,23,4,пятница,74.0,23.0,40.666667,39.696429
997,2023-06-03 00:00:00,28,0,5,суббота,29.0,6.0,40.916667,39.654762
998,2023-06-03 01:00:00,15,1,5,суббота,28.0,1.0,41.833333,39.666667
